In [ ]:
import os
import cv2
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical

# Function to load images from a folder
def load_images_from_folder(folder, target_size=(100, 100)):
    images = []
    labels = []
    for category in os.listdir(folder):
        category_folder = os.path.join(folder, category)
        for filename in os.listdir(category_folder):
            img = cv2.imread(os.path.join(category_folder, filename))
            if img is not None:
                img_resized = cv2.resize(img, target_size)  # Resize image to a fixed size
                images.append(img_resized)
                labels.append(category)  # Assuming folder names are the labels
    return np.array(images), np.array(labels)

# Load images from a folder
train_folder = 'dataset/train'
test_folder = 'dataset/test'
train_images, train_labels = load_images_from_folder(train_folder)
test_images, test_labels = load_images_from_folder(test_folder)

# Normalize pixel values to range [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Convert labels to one-hot encoding
label_encoder = {}
for i, label in enumerate(set(np.concatenate([train_labels, test_labels]))):
    label_encoder[label] = i
train_labels_encoded = [label_encoder[label] for label in train_labels]
test_labels_encoded = [label_encoder[label] for label in test_labels]
train_labels_one_hot = to_categorical(train_labels_encoded)
test_labels_one_hot = to_categorical(test_labels_encoded)

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(set(train_labels)), activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels_one_hot, epochs=10, validation_data=(test_images, test_labels_one_hot))

# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels_one_hot)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')